In [ ]:
#you need to install these
%pip install dspy
%pip install ipywidgets
%pip install IPython
%pip install requests



In [25]:
#import
import dspy
import os
import dotenv
import wget
import requests

download the context, in this case the minetuning readme

In [ ]:
url = "https://raw.githubusercontent.com/rawwerks/MineTuning/main/README.md"
response = requests.get(url)
context = response.text
with open(context_file, 'r') as file:
    context = file.read()
    
from IPython.display import Markdown
display(Markdown(context))

we can use multiple generations (`count`) to speed up the process. 
todo: first get it working with count = 1, then figure out how to reconcile multiple generations with multiple attempts.

In [20]:
#setup dspy
dotenv.load_dotenv(os.path.expanduser("~/.env"))  # load OpenAI API key from .env file, or you can set directly with 
llm = dspy.OpenAI(model='gpt-4o', max_tokens=4096, temperature=0.5)
#llm = dspy.OpenAI(model='gpt-4o', max_tokens=4096, api_key="sk-") #or you can set the API key directly here
dspy.settings.configure(lm=llm)

count = 2

query = input("Query:") #from interactive input
#query = "steampunk chic superheroes." #alternatively write the string directly into the code

#signature
class Visualization(dspy.Signature):
    """You are an expert who is always learning and improving based on user feedback."""
    query = dspy.InputField(description="this is the user's query.")
    context = dspy.InputField(description="this is the context of the query. study it carefully.")
    rationale = dspy.OutputField(description="this is your rationale for your response.")
    response = dspy.OutputField(description="this is your response.")
    history = dspy.InputField(required=False, description="this is the history of the conversation with the user. use it to improve your next response.")

#module 
class VisualizerModule(dspy.Module):
    def __init__(self, **kwargs):
        super().__init__()
        self.signature = Visualization
        self.predictor = dspy.ChainOfThought(self.signature, n=count)

    def forward(self, query, context, score=None, feedback=None):
        result = self.predictor(query=query, context=context, score=score, feedback=feedback, num_generations=count)
        #print(result.completions)
        return [dspy.Prediction(rationale=completion.rationale, response=completion.response) for completion in result.completions]

#instance
visualizer = VisualizerModule()

response = visualizer(query=query, context=context)
display(response)


[Prediction(
     rationale='Query: i\'m confused. eli5\n\nContext:\n# Mine-Tuning\n\nMine-tuning is a methodology for synchronizing human and AI attention. Mine-tuning is a state-of-mind(s), an approach to continuous learning and improvement through on-the-job feedback. Mine-tuning offers a rapid approach to customizing AI programs to our personal preferences, professions, and priorities.\n\n**TL;DR - Mine-tuning makes it easy for humans to teach AI how to do a good job.**\n\n## Key Principles of Mine-tuning: \n\n1. **On-the-job learning.** In contrast to traditional fine-tuning methods which separate "learning" from "doing", the AI program is continuously trained on the user\'s feedback -- while assisting the user in the actual task at hand.\n2. **Immediate human-in-the-loop feedback.** Attention is all it takes. The AI program is given immediate human feedback, preferably on every response.\n3. **Bootstrap to mastery.** Training data sets can be built up one example at a time. At th

here, we are going to calculate the `score` to be the reciprocal of the number of attempts to achieve a 5 star review. for all responses with less than 5 stars, the score is 0. 

In [ ]:
def get_user_rating(response):
    while True:
        try:
            rating = int(input(f"{response}\n---\nPlease rate on a 5-point scale (1-5): "))
            if 1 <= rating <= 5:
                return rating  # Cast the integer to a string before returning
            else:
                print("Invalid input. Please enter a number between 1 and 5.")
        except ValueError:
            print("Invalid input. Please enter a number.")

def get_user_feedback(response):
    feedback = input(f"{response}\n---\nPlease provide your feedback: ")
    return feedback

def get_score(attempts, rating):
    if rating >= 5:
        score = 1/attempts
    else:
        score = 0
    print("calculated score: " + str(score) + " based on rating: " + str(rating) + " and attempts: " + str(attempts))
    return score

history = []

# Get rating and feedback for each response
for response in response.completions:
    while True:
        print(response)
        rating = get_user_rating(response)
        feedback = get_user_feedback(response)
        this_example = {
            'query': query,
            #'context': context, #this is omitted for clarity, but might be useful to include if the context were not static. (ie, RAG)
            'rationale': response.rationale,
            'response': response,
            'rating': rating,
            'feedback': feedback
        }
        history.append(this_example)
        if rating >= 5:
            print("history:")
            print(history)
            break
        history_str = ' '.join(str(item) for item in history)  # Convert each item to string before joining
        attempt_count += 1

